<a href="https://colab.research.google.com/github/nullpitch-dev/DS_L1_Notebooks/blob/master/DS_L1_EX_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

# loading data from a csv fle
url = 'https://raw.githubusercontent.com/nullpitch-dev/hj_public/master/Womens_Clothing_Reviews.csv'
data = pd.read_csv(url)

# Pre-processing
data = data.dropna(subset={'Seq', 'ClothingID', 'Age', 'TitleLength',
                           'ReviewTextLength', 'Rating'})

def ageClass(x):
    if (x >= 20) & (x < 30):
        return 'A'
    elif (x >= 30) & (x < 40):
        return 'B'
    elif (x >= 40) & (x < 50):
        return 'C'
    elif (x >=50) & (x < 60):
        return 'D'
    else:
        return 'E'

data['ageCD'] = data['Age'].apply(lambda x: ageClass(x))
data['Pos_Neg'] = data['Rating'].apply(lambda x: 1 if x > 4 else 0)



In [0]:
# [1]

data1 = data[data['DepartmentName'] != '0']

mean1 = data1.groupby('DepartmentName').mean()['Rating']
mean1 = mean1.reset_index()
std1 = data1.groupby('DepartmentName').std()['Rating']
std1 = std1.reset_index()

std_mean = pd.merge(left=std1, right=mean1, on='DepartmentName',
                    suffixes=('_std', '_mean'))
std_mean['CV'] = std_mean['Rating_std'] / std_mean['Rating_mean']
std_mean.sort_values('CV', ascending=False, inplace=True)
std_mean.reset_index(inplace=True)

cv = std_mean['CV'][0]
depart = std_mean['DepartmentName'][0]

print(f'Answer [1] : Department = {depart}, CV = {cv:.2f}')

Answer [1] : Department = Trend, CV = 0.35


In [0]:
# [2]

import math

temp = data.groupby('ClothingID').count()
temp.reset_index(inplace=True)
temp = temp[temp['Seq'] >= 30]
clothings = list(temp['ClothingID'])

data2 = data[data['ClothingID'].isin(clothings)]

temp2 = data2.groupby('ClothingID').agg({'Seq': 'count', 'Pos_Neg': 'sum'})
temp2.reset_index(inplace=True)
temp2 = temp2[['ClothingID', 'Seq', 'Pos_Neg']]

temp2['Rate'] = temp2['Pos_Neg'] / temp2['Seq']
temp2.sort_values(by='Rate', ascending=False, inplace=True)

clothingID = temp2.iloc[1][0]
rate = math.floor(temp2.iloc[1][3] * 1000) / 1000

print(f'Answer [2] : ClothingID = {int(clothingID)}, Positive Rate = {rate}')

Answer [2] : ClothingID = 964, Positive Rate = 0.794


In [0]:
# [3] Using f_oneway

from scipy import stats
import math

data3_A = data[data['ageCD'] == 'A']['Rating']
data3_B = data[data['ageCD'] == 'B']['Rating']
data3_C = data[data['ageCD'] == 'C']['Rating']
data3_D = data[data['ageCD'] == 'D']['Rating']
data3_E = data[data['ageCD'] == 'E']['Rating']

f_value, p_value = stats.f_oneway(data3_A, data3_B, data3_C, data3_D, data3_E)

print(f'Answer [3] : F-value = {math.floor(f_value * 10000) / 10000}')

Answer [3] : F-value = 8.4989


In [0]:
# [3] Using ols

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import math

results = ols(formula='Rating ~ C(ageCD)', data=data).fit()
anova_table = anova_lm(results)

f_value = anova_table['F'][0]

print(f'Answer [3] : F-value = {math.floor(f_value * 10000) / 10000}')

Answer [3] : F-value = 8.4989


In [0]:
# [4]

from sklearn.linear_model import LogisticRegression

data4 = data[data['ClassName'] != '0']
data4 = pd.get_dummies(data4, columns={'ageCD'}, drop_first=True)

def getPrevRating(x):
    if len(data4[data4['Seq'] == x - 1]) == 0:
        return data4[data4['Seq'] == x]['Rating'].values[0]
    else:
        return data4[data4['Seq'] == x - 1]['Rating'].values[0]

data4['prior_Rating'] = data4['Seq'].apply(lambda x: getPrevRating(x))

train4 = data4[data4['ClassName'] != 'Intimates']
test4 = data4[data4['ClassName'] == 'Intimates']

x_cols = ['Rating', 'ageCD_B', 'ageCD_C', 'ageCD_D', 'ageCD_E', 'TitleLength',
          'ReviewTextLength', 'prior_Rating']
train_x4 = train4[x_cols]
train_y4 = train4['RecommendedIND']
test_x4 = test4[x_cols]
test_y4 = test4['RecommendedIND']

lr = LogisticRegression(C=100000, random_state=1234, penalty='l2', solver='newton-cg')
model = lr.fit(train_x4, train_y4)
pred = model.predict_proba(test_x4)

pred_df = pd.DataFrame(pred, columns=['N', 'Y'])
pred_df['Fact'] = test_y4.reset_index()['RecommendedIND']
pred_df['Pred'] = pred_df['Y'].apply(lambda x: 'Y' if x > 0.5 else 'N')

pred_df['Accuracy'] = (pred_df['Fact'] == pred_df['Pred'])
accuracy = pred_df['Accuracy'].sum() / pred_df['Accuracy'].count()

print(f'Answer [4] : Accuracy = {accuracy:.3f}')

Answer [4] : Accuracy = 0.946
